# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [44]:
from selenium import webdriver # Criar navegador
from selenium.webdriver.common.by import By # Localizar elementos (os itens de um site)
from selenium.webdriver.common.keys import Keys # Permite clicar teclas no teclado

navegador = webdriver.Chrome()

# Passo 1 - Entrar no google
navegador.get("http://google.com/")

# Passo 2 - Pesquisar cotacao do dolar
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Passo 3 - Pegar a cotacao do dolar
cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# Passo 4 - Pegar a cotacao do Euro
navegador.get("http://google.com/")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação Euro")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# Passo 5 - Pegar a cotacao do Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

5.138
5.409260710000001
306.28


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [45]:
# Passo 6 - Atualizar a minha base de dados com as novas cotacoes
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.138191,5138.139618,1.40,7193.395465
1,Carro Renault,4500.00,Euro,5.402818,24312.681504,2.00,48625.363008
2,Notebook Dell,899.99,Dólar,5.138191,4624.320518,1.70,7861.344881
3,IPhone,799.00,Dólar,5.138191,4105.414609,1.70,6979.204835
4,Carro Fiat,3000.00,Euro,5.402818,16208.454336,1.90,30796.063238
5,Celular Xiaomi,480.48,Dólar,5.138191,2468.798012,2.00,4937.596023
6,Joia 20g,20.00,Ouro,306.280000,6125.600000,1.15,7044.440000


- Atualizando os preços e o cálculo do Preço Final

In [46]:
# Atualizar a cotacao de acordo com a moeda correspondente
# Dolar - # As linhas onde a coluna "Moeda" = "Dolar"
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)

# Euro - # As linhas onde a coluna "Moeda" = "Euro"
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)

# Ouro - # As linhas onde a coluna "Moeda" = "Ouro"
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# Atualizar o preco de compra = preco original * cotacao
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# Atualizar o preco de venda = preco de compra * margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.138000,5137.948620,1.40,7193.128068
1,Carro Renault,4500.00,Euro,5.409261,24341.673195,2.00,48683.346390
2,Notebook Dell,899.99,Dólar,5.138000,4624.148620,1.70,7861.052654
3,IPhone,799.00,Dólar,5.138000,4105.262000,1.70,6978.945400
4,Carro Fiat,3000.00,Euro,5.409261,16227.782130,1.90,30832.786047
5,Celular Xiaomi,480.48,Dólar,5.138000,2468.706240,2.00,4937.412480
6,Joia 20g,20.00,Ouro,306.280000,6125.600000,1.15,7044.440000


### Agora vamos exportar a nova base de preços atualizada

In [48]:
tabela.to_excel("Produtos.xlsx", index=False)
navegador.quit()